In [1]:
import requests
from synapse.ml.fabric import FabricCredential
 
token = FabricCredential().get_token()
headers = dict()
headers['Authorization'] = f'Bearer {token.token}'

StatementMeta(, f09da00e-4c95-4ea9-be24-e79b4b8c4f4f, 3, Finished, Available, Finished)

#### Invoke and monitor pipeline execution

In [ ]:
import requests
from urllib.parse import urlparse
import time
import json
from concurrent.futures import ThreadPoolExecutor, as_completed

# Replace with your workspace and pipeline identifiers
workspace = ""
pipeline = " "

url = f"https://api.fabric.microsoft.com/v1/workspaces/{workspace}/items/{pipeline}/jobs/instances?jobType=Pipeline"

payload = {
    "executionData": {}
}


def execute_pipeline():

    response = requests.post(url, json=payload, headers=headers)
    response.raise_for_status()

    if response.status_code in [200, 202]:
        location_header = response.headers.get("Location")
        parsed_url = urlparse(location_header)
        path_parts = parsed_url.path.split('/')
        job_instance_id = path_parts[-1]

        url_get = f"https://api.fabric.microsoft.com/v1/workspaces/{workspace}/items/{pipeline}/jobs/instances/{job_instance_id}"

        while True:
            # Make the GET request
            response_status = requests.get(url_get, headers=headers)

            if response_status.status_code == 200:
                try:
                    response_data = response_status.json()
                    status = response_data.get('status')

                    if status == 'Completed':
                        print("Status:", status)
                        break  # Exit the loop if status is "Completed"
                    else:
                        print("Execution is still in progress. Checking again...")
                        print("Status:", status)
                except json.JSONDecodeError:
                    print("JSON Decode Error. Retrying...")

            else:
                print('Error:', response_status.status_code)
                break

            time.sleep(20)

def main():
    with ThreadPoolExecutor(max_workers=15) as executor:
        futures = [executor.submit(execute_pipeline) for _ in range(15)]

        for future in as_completed(futures):
            try:
                future.result()
            except Exception as e:
                print(f"Pipeline execution generated an exception: {e}")

if __name__ == "__main__":
    main()
